In diesem Notebook wird die Pipeline aufgebaut.

In [1]:
%load_ext autoreload
%autoreload 2

import os
if os.getcwd() == '/home/jovyan/work': # jhub
    os.chdir("24FS_I4DS27/main/") 
    os.system("make reqs")
else: # local
    os.chdir("../")

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

from src.data.mri import MRIDataModule
from src.data.covidx import COVIDXDataModule
from src.utils.download import download_models
from src.utils.transform_perturbation import AddImagePerturbation
from src.utils.uap_helper import generate_adversarial_images_from_model_dataset, get_model
from src.utils.adv_training import pipeline, get_transform
from src.models.imageclassifier import ImageClassifier

from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.figsize"] = (16, 8)

torch.set_float32_matmul_precision('high')

In [3]:
ENTITY = "24FS_I4DS27"
PROJECT = "baselines"
NUM_WORKERS = 8

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [4]:
# Download models if not present
models = download_models(ENTITY, PROJECT)

In [5]:
modelname, dataset = "resnet50", "mri_data"
print(f"\n---\nModel: {modelname} - Dataset: {dataset}")
print(f"Device: {device}")


lr = 1e-0
max_retries = 10
for attempt in range(max_retries):
    try:
        pipeline(
            modelname=modelname,
            dataset=dataset,
            n_robustifications=10,
            i=5,
            n=200,
            t=50,
            p=2,
            lambda_norm=0.05,
            r=0.2,
            eps=1e-6,
            lr_uap=lr,
            seed=42,
            num_workers=NUM_WORKERS,
            device=device,
            verbose=False,
        )
    # CUDA out of memory error
    except RuntimeError as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        if attempt >= max_retries - 1:
            print("All retries failed, lowering learning rate")
            lr /= 10**0.5


---
Model: resnet50 - Dataset: mri_data
Device: cuda


Universal Pertubation:   0%|          | 0/5 [00:00<?, ?it/s]

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs


...

HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                                                    | 0/? [00:00<?, …

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_BinaryAUROC      │    0.9222277402877808     │
│    test_BinaryAccuracy    │    0.7385786771774292     │
│    test_BinaryF1Score     │    0.7858628034591675     │
│   test_BinaryPrecision    │         0.984375          │
│     test_BinaryRecall     │    0.6539792418479919     │
│  test_BinarySpecificity   │    0.9714285731315613     │
│         test_loss         │    0.9133037328720093     │
└───────────────────────────┴───────────────────────────┘

Universal Pertubation:   0%|          | 0/5 [00:00<?, ?it/s]